In [1]:
from llamaapi import LlamaAPI
import os
import nltk

from Document import Document

print("=== Downloading necessary NLTK pakages, if not already present")

d1 = nltk.download("punkt", quiet=True)
d2 = nltk.download("stopwords", quiet=True)
d3 = nltk.download("wordnet", quiet=True)


_MODEL = "llama-13b-chat"
_CONTEXT_SIZE = 128  # In MB

# A bit of obfuscation to make crawlers' life miserable
# It's just the a.pi k.ey, written so that it's not a clear string
# also, `LlamaAPI' is decoupled from the actual string so as to make it even harder to parse it

_a1 = "AbZ".split("b")[1]
_a2 = "BSaNbNTlp0o0bILov9Z3U7XmnP4DhwrV24jgq"
_a3 = "A7kX0SPThAArXd0jNZxQ2WZ"
_call = lambda x: LlamaAPI(x + _a1)

llama = _call(f"LL-{_a3}2l1{_a2}")


print(f"=== Loaded\nUsing model ``{_MODEL}``")

=== Downloading necessary NLTK pakages, if not already present
=== Loaded
Using model ``llama-13b-chat``


In [2]:
def get_MB_size(path):
    file_stats = os.stat(path)
    return file_stats.st_size / (1024 * 1024)

In [3]:
path = "./test.txt"

if get_MB_size(path) > _CONTEXT_SIZE:
    print("Over the limit")
document = Document(path).make_bow()
# document.counts()
text = document.text(escape=True)
text

'\\\n\\ \\\n\\ \\\n\\ File:T4\\ Bacteriophage\\.gifFile:PhageExterior\\.svgFile:11\\ Hegasy\\ Phage\\ T4\\ Wiki\\ E\\ CCBYSA\\.png\\\n\\ A\\ bacteriophage\\ \\(\\)\\ also\\ known\\ informally\\ as\\ a\\ phage\\ \\(\\)\\ is\\ a\\ virus\\ that\\ infects\\ and\\ replicates\\ within\\ bacteria\\ and\\ archaea\\.\\ The\\ term\\ was\\ derived\\ from\\ "bacteria"\\ and\\ the\\ Greek\\ language\\\n\\ Bacteriophages\\ are\\ among\\ the\\ most\\ common\\ and\\ diverse\\ entities\\ in\\ the\\ biosphere\\.\\ Bacteriophages\\ are\\ ubiquitous\\ viruses\\ found\\ wherever\\ bacteria\\ exist\\.\\ It\\ is\\ estimated\\ there\\ are\\ more\\ than\\ 10<sup>31</sup>\\ bacteriophages\\ on\\ the\\ planet\\ more\\ than\\ every\\ other\\ organism\\ on\\ Earth\\ including\\ bacteria\\ combined\\.\\ Viruses\\ are\\ the\\ most\\ abundant\\ biological\\ entity\\ in\\ the\\ water\\ column\\ of\\ the\\ worlds\\ oceans\\ and\\ the\\ second\\ largest\\ component\\ of\\ biomass\\ after\\ prokaryotes\\ where\\ up\\ to\

In [4]:
# API Request JSON Cell
api_request_json = {
    "model": _MODEL,
    "messages": [
        {"role": "system", "content": "You are a serious assistant"},
        {
            "role": "user",
            "content": text,
        },
    ],
}

# Make your request and handle the response
response = llama.run(api_request_json)
resp = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
print(resp["choices"][0]["message"]["content"])

"Nel mezzo del cammin di nostri mortali giorni,
mingendo con la mia arte, che alla misura
del mio bel poco sia giunta, e son d'oggi il libro" (Inferno, canto primo, versetti 1-3).
